<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/Gradio%E3%81%AEAPI%E3%82%92%E7%94%A8%E3%81%84%E3%81%9FNDL%E5%8F%A4%E5%85%B8%E7%B1%8DOCR%E3%81%AE%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GradioのAPIを用いたNDL古典籍OCRの例

## 関数

In [ ]:
import base64
from urllib import request
import PIL.Image
import requests

def image_to_data_url(filename):
    """画像をbase64に変換する"""
    ext = filename.split('.')[-1]
    prefix = f'data:image/{ext};base64,'
    with open(filename, 'rb') as f:
        img = f.read()
    return prefix + base64.b64encode(img).decode('utf-8')

def convertUrl2Base64(url, local=False):
    """URLをbase64に変換する"""
    if local:
        path = url
    else:
        path = url.split("/")[-1].replace(".bmp", ".jpg")
        request.urlretrieve(url, path)

    d = image_to_data_url(path)

    return d, PIL.Image.open(path)

## 実行

例として、「源氏物語」（国立国会図書館所蔵）を使用しています。

In [ ]:
# 変数
url = "https://dl.ndl.go.jp/api/iiif/2585098/R0000003/full/800,/0/default.jpg"
# url = "https://images.metmuseum.org/CRDImages/as/original/DP121536.jpg"
endpoint = "https://nakamura196-ndl-kotenseki-ocr.hf.space/run/predict"

# メイン
d, img = convertUrl2Base64(url)
r = requests.post(url=endpoint, json={"data": [d]})

if r.status_code == 200:
  res = r.json()["data"][1] # 配列の2番目にJSONデータが格納されます。
else:
  res = {
      "status_code": r.status_code,
      "text": r.text
  }

res